In [6]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(0, "C:/Users/16221/Desktop/Courses/Project Lab 2/Factor-Research/src")
from Features import fundamental_features_new, fundamental_features, technical_features
import warnings
warnings.filterwarnings("ignore") 

In [7]:
zacks_fc = pd.read_csv("../data/fundamental_data/ZACKS_FC.csv", delimiter=',')
zacks_fc['per_end_date'] = pd.to_datetime(zacks_fc['per_end_date'])
zacks_fc['per_end_date'] = zacks_fc['per_end_date'].dt.tz_localize('US/Eastern')
zacks_fc['filing_date'] = pd.to_datetime(zacks_fc['filing_date'])
zacks_fc['filing_date'] = zacks_fc['filing_date'].dt.tz_localize('US/Eastern')
zacks_fc = zacks_fc.sort_values('per_end_date').set_index('per_end_date')

zacks_fr = pd.read_csv("../data/fundamental_data/ZACKS_FR.csv", delimiter=',')
zacks_fr['per_end_date'] = pd.to_datetime(zacks_fr['per_end_date'])
zacks_fr['per_end_date'] = zacks_fr['per_end_date'].dt.tz_localize('US/Eastern')
zacks_fr = zacks_fr.sort_values('per_end_date').set_index('per_end_date')

data = {
    "zacks_fc": zacks_fc,
    "zacks_fr": zacks_fr
}

In [9]:
tickers_df = pd.read_csv("../data/sp100_cleaned.csv")  # 假设该文件中有一列 'ticker'
tickers = tickers_df['ticker'].tolist()

In [10]:
combined_features = []
for ticker in tickers[:100]:
    data = {
    "zacks_fc": zacks_fc,
    "zacks_fr": zacks_fr
    }
    try:
        print(f"Processing ticker: {ticker}")
        obj = fundamental_features_new.BuildFeatures(data, ticker)
        obj.process_financial_statements()
        obj.extend_monthly()
        
        tech = technical_features.BuildFeatures(ticker)
        tech.build_technical_features()
        tech.technical_features.index = tech.technical_features.index.tz_convert('US/Eastern')

        df = pd.merge_asof(obj.fundamental_features, tech.technical_features, 
                           left_index=True, right_index=True, direction='nearest')

        df['ticker'] = ticker  
        
        combined_features.append(df)
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

Processing ticker: VRSK
Error: Moving_Average_Convergence/Divergence_Fix_12/26
Error processing VRSK: Merge keys contain null values on left side
Processing ticker: EVTC
Error: Moving_Average_Convergence/Divergence_Fix_12/26
Error processing EVTC: Merge keys contain null values on left side
Processing ticker: AA
Error: Moving_Average_Convergence/Divergence_Fix_12/26
Error processing AA: Merge keys contain null values on left side
Processing ticker: PFG
Error: Moving_Average_Convergence/Divergence_Fix_12/26
Processing ticker: NXPI
Error: Moving_Average_Convergence/Divergence_Fix_12/26
Error processing NXPI: Merge keys contain null values on left side
Processing ticker: CELH
Error: Moving_Average_Convergence/Divergence_Fix_12/26
Error processing CELH: Merge keys contain null values on left side
Processing ticker: AXP
Error: Moving_Average_Convergence/Divergence_Fix_12/26
Processing ticker: VZ
Error: Moving_Average_Convergence/Divergence_Fix_12/26
Processing ticker: BA
Error: Moving_Avera

In [11]:
combined_features

[                           total revenue    ebitda   eps  \
 filing_date                                                
 2006-04-30 20:00:00-04:00         2402.2  467.3998  1.01   
 2006-05-31 20:00:00-04:00         2402.2  467.3998  1.01   
 2006-06-30 20:00:00-04:00         2402.2  467.3998  1.01   
 2006-07-31 20:00:00-04:00         2459.8  386.5001  0.76   
 2006-08-31 20:00:00-04:00         2459.8  386.5001  0.76   
 ...                                  ...       ...   ...   
 2024-05-31 20:00:00-04:00         4053.3  627.0999  2.22   
 2024-06-30 20:00:00-04:00         4310.8  443.8999  1.49   
 2024-07-31 20:00:00-04:00         4310.8  443.8999  1.49   
 2024-08-31 20:00:00-04:00         4310.8  443.8999  1.49   
 2024-09-30 20:00:00-04:00         3011.6 -293.7998 -0.95   
 
                            Dividend Payout Ratio  total_current_assets      \
 filing_date                                                                  
 2006-04-30 20:00:00-04:00                    N

In [12]:
# 4. Merge all stock data
final_df = pd.concat(combined_features)
final_df = final_df.sort_index()
print("Combined DataFrame shape:", final_df.shape)

# 5. Data preprocessing and handling missing values
print("Before processing missing values, final_df shape:", final_df.shape)
print("Missing values per column:")
print(final_df.isnull().sum())

# Use forward fill and backward fill to fill in missing values
final_df = final_df.ffill().bfill()
print("\nAfter forward fill and backward fill, final_df shape:", final_df.shape)
print("Missing values per column:")
print(final_df.isnull().sum())

# Drop columns that are entirely missing
final_df = final_df.dropna(axis=1, how='all')
print("\nAfter dropping columns with all missing values, final_df shape:", final_df.shape)
print("Missing values per column:")
print(final_df.isnull().sum())

# Drop any remaining rows that contain missing values
final_df = final_df.dropna()
print("\nAfter dropping rows with any missing values, final_df shape:", final_df.shape)

if final_df.shape[0] == 0:
    raise ValueError("The dataset is empty. Please check the data merging and missing value handling steps!")

# 6. Select numerical data for PCA (non-numerical columns, such as 'ticker', will be excluded)
final_df_numeric = final_df.select_dtypes(include=[np.number])
print("\nNumeric columns used for PCA:")
print(final_df_numeric.columns)
final_df = final_df_numeric

# 7. Split data into training and testing sets (80% training, 20% testing)
train_data, test_data = train_test_split(final_df, test_size=0.2, random_state=42, shuffle=True)
print("\nTraining data shape:", train_data.shape)
print("Testing data shape:", test_data.shape)

# 8. Standardize data and perform PCA
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_data)
test_scaled = scaler.transform(test_data)

# Choose to extract 5 principal components (adjust as needed)
n_components = 5
pca = PCA(n_components=n_components)
pca.fit(train_scaled)

# Output the explained variance ratio for each principal component
print("\n" + "="*80)
print("Explained Variance Ratio on Training Data:")
for i, ratio in enumerate(pca.explained_variance_ratio_):
    print(f"  PC{i+1}: {ratio:.4f}")
print("="*80)

# Output the principal components (loadings) with their corresponding features
print("\nPrincipal Components (Loadings):")
for i, component in enumerate(pca.components_):
    loadings = pd.Series(component, index=train_data.columns)
    loadings_sorted = loadings.sort_values(ascending=False)
    print(f"\nPC{i+1} Loadings:")
    print(loadings_sorted.to_string())
    print("-" * 80)

# Transform the test data into the PCA space
test_pca = pca.transform(test_scaled)

# Compute the reconstruction error on the test data via inverse transformation
reconstructed_test = pca.inverse_transform(test_pca)
total_variance = np.var(test_scaled, axis=0).sum()
reconstruction_error = np.var(test_scaled - reconstructed_test, axis=0).sum()
variance_explained_test = 1 - reconstruction_error / total_variance

print("\n" + "="*80)
print("{:^80}".format("PCA RESULTS"))
print("="*80)
print(f"Overall variance explained on Test Data by {n_components} principal components: {variance_explained_test:.4f}")
print("="*80)


Combined DataFrame shape: (13388, 165)
Before processing missing values, final_df shape: (13388, 165)
Missing values per column:
total revenue                             0
ebitda                                  445
eps                                     127
Dividend Payout Ratio                 13388
total_current_assets                      0
                                      ...  
Midpoint_Price_over_period              142
Lowest_value_over_period                142
Index_of_lowest_value_over_period         0
Highest_value_over_period               142
Index_of_highest_value_over_period        0
Length: 165, dtype: int64

After forward fill and backward fill, final_df shape: (13388, 165)
Missing values per column:
total revenue                             0
ebitda                                    0
eps                                       0
Dividend Payout Ratio                 13388
total_current_assets                      0
                                      ...  
Mi